<a href="https://colab.research.google.com/github/NINGTANG1124/UPF-HFI/blob/main/notebooks/0802_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# connect googledrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Read intake data (including Descriptionen and FoodGroupen)
import pandas as pd
import re

file_path = "/content/drive/MyDrive/UPF-HFI/Bradford_original data/1. Dietmasterfile_foodlevel_clean.xls"
intake_df = pd.read_excel(file_path)

# Define text cleaning function
def clean_text(col):
    return col.astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

# Apply to key fields
intake_df["Foodgroupen_clean"] = clean_text(intake_df["Foodgroupen"])
intake_df["Descriptionen_clean"] = clean_text(intake_df["Descriptionen"])

# 添加 Subgroupcode 的清洗列
intake_df["Subgroupcode_clean"] = clean_text(intake_df["Subgroupcode"])


In [3]:
# 设置 att3 文件路径
att3_path = "/content/drive/MyDrive/UPF-HFI/nova matching files/att3-excel.xlsx"

# 读取 att3 文件
att3 = pd.read_excel(att3_path)

# 清洗 group code（假设列名为 'Subsidiary food group code'）
att3['code_clean'] = att3['Subsidiary food group code'].astype(str).str.upper().str.strip()

# 创建字典（包括 *）
group_to_nova = dict(zip(att3['code_clean'], att3['NOVA food group']))

# 清洗 intake 数据的 Subgroupcode 列
intake_df['Subgroupcode_clean'] = intake_df['Subgroupcode'].astype(str).str.upper().str.strip()


In [4]:
# Step 1: 映射 group code 到 NOVA_step1
intake_df['NOVA_step1'] = intake_df['Subgroupcode_clean'].map(group_to_nova)

# Step 2: 添加匹配说明列（仅记录成功匹配的）
intake_df['match_reason_step1'] = intake_df['NOVA_step1'].apply(
    lambda x: 'group code match' if str(x).isdigit() else None
)

# 哪些行已经匹配成功？多少还没有？
# 匹配情况预览
intake_df['NOVA_step1'].value_counts(dropna=False)


,count
NOVA_step1,
4,8626
*,7643
1,4360
3,483
**,414
NaN,356
2,335


In [5]:
# 导出检查文件（只保留关键列）
cols_to_export = [
    'Descriptionen', 'Foodgroupen',
    'Subgroupcode', 'Subgroupcode_clean',
    'NOVA_step1', 'match_reason_step1'
]

check_df = intake_df[cols_to_export].copy()
check_df.to_excel("/content/drive/MyDrive/UPF-HFI/0728outcome/step1.xlsx", index=False)


# 精准提取 Step 2 目标项（NOVA_step1 是 * 或 **）

In [6]:
# Step 2：提取目标记录（NOVA 是 * 或 **）
df_step2_targets = intake_df[intake_df['NOVA_step1'].isin(["*", "**"])].copy()

# 按 Subgroupcode_clean 去重，只保留每组一个代表行
df_step2_unique = df_step2_targets.drop_duplicates(subset="Subgroupcode_clean")

# 选择导出的关键列
cols_step2 = [
    'Descriptionen', 'Foodgroupen',
    'Subgroupcode', 'Subgroupcode_clean',
    'NOVA_step1', 'match_reason_step1'
]
df_step2_unique_simple = df_step2_unique[cols_step2]

# 导出为 Excel 文件
df_step2_unique_simple.to_excel("/content/drive/MyDrive/UPF-HFI/0728outcome/step2_groupcodes_unique.xlsx", index=False)


# 提取NaN


In [10]:
# 创建一个新的辅助列 Subgroupcode_clean_na_safe，仅用于提取 NaN 行
intake_df['Subgroupcode_clean_na_safe'] = (
    intake_df['Subgroupcode']
    .where(intake_df['Subgroupcode'].notna())  # 保留真正的 NaN
    .astype(str).str.upper().str.strip()
)

# 提取 NOVA_step1 是 NaN 的行
df_step1_nan = intake_df[intake_df['NOVA_step1'].isna()].copy()

# 导出相关字段（保留原有清洗列 Subgroupcode_clean 和新建列）
cols_nan = [
    'Descriptionen', 'Foodgroupen', 'Subgroupcode',
    'Subgroupcode_clean', 'Subgroupcode_clean_na_safe',
    'NOVA_step1', 'match_reason_step1'
]

df_step1_nan[cols_nan].to_excel(
    "/content/drive/MyDrive/UPF-HFI/0728outcome/step1_nan_records_safe.xlsx", index=False
)


In [9]:
# 看一下 NaN 的行实际是什么 code
print(df_step1_nan['Subgroupcode_clean'].unique())

# 对比 att3 中是否有这些
print(att3['code_clean'].unique())


['62R' 'NAN']
['1C' '1D' '1E' '1F' '1G' '1R' '2R' '3R' '4R' '5R' '6R' '7A' '7B' '8B'
 '8C' '8D' '8E' '9C' '9D' '9E' '9F' '9G' '9H' '10R' '11R' '12R' '13A'
 '13B' '13R' '14A' '14B' '14R' '15B' '15C' '15D' '16C' '16D' '17R' '18A'
 '18B' '19A' '19R' '20A' '20B' '20C' '21A' '21B' '22A' '22B' '23A' '23B'
 '24A' '24B' '25A' '25B' '26A' '27A' '27B' '28R' '29R' '30A' '30B' '31A'
 '31B' '32A' '32B' '33R' '34C' '34D' '34E' '34F' '34G' '34H' '35A' '35B'
 '36A' '36B' '36C' '37A' '37B' '37C' '37D' '37E' '37F' '37I' '37K' '37L'
 '37M' '38A' '38C' '38D' '39A' '39B' '40A' '40B' '40C' '40D' '40E' '40R'
 '41A' '41B' '41R' '42R' '43R' '44R' '45R' '47A' '47B' '48A' '48B' '48C'
 '49A' '49B' '49C' '49D' '49E' '50A' '50C' '50D' '50E' '50R' '51A' '51B'
 '51C' '51D' '51R' '52A' '52R' '53R' '54A' '54B' '54C' '54D' '54E' '54F'
 '54G' '54H' '54I' '54J' '54K' '54L' '54M' '54N' '54P' '55R' '56R' '57A'
 '57B' '57C' '58A' '58B' '58C' '59R' '60R' '61R']
